# **1. Data Cleaning**


In [2]:
import pandas as pd
import yaml

# Load configuration from YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

df = pd.read_csv(config['input_file'], sep="\t")
print(df.shape)
print(df.columns)
df.head()

(26210, 53)
Index(['isbn', 'baf', 'first_version', 'second_version', 'final_version',
       'FNF', 'VJ', 'title', 'subtitle', 'publishing_date',
       'publishing_date_original', 'title_original', 'language_original',
       'series_name', 'series_part', 'series_index', 'edition', 'publisher',
       'pages', 'num_pages', 'language_original_name',
       'language_original_adjective_neuter', 'language_original_adjective_mf',
       'is_translation', 'is_title_original_different', 'translator',
       'author_name', 'author_surname', 'pseudonym', 'birthdate', 'birth_year',
       'deathdate', 'death_year', 'death_age', 'publishing_age', 'birthplace',
       'deathplace', 'nationalities', 'nationality_adjective_mf',
       'nationality_adjective_neuters', 'nationality_country_names', 'awards',
       'bibliography', 'bibliography_place', 'countries_published',
       'known_works', 'product_form', 'debut', 'debut_year', 'author_status',
       'title_status', 'professions', 'blurb'],
 

C:\Users\kevin\AppData\Local\Temp\ipykernel_1156\2166009631.py:8: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(config['input_file'], sep="\t")


,isbn,baf,first_version,second_version,final_version,FNF,VJ,title,subtitle,publishing_date,...,bibliography_place,countries_published,known_works,product_form,debut,debut_year,author_status,title_status,professions,blurb
0,9789089657633,mas_baf_processed_26320,"Een boek over duurzaamheid, duurzaam leven en ...",Een boek over duurzaam beleid binnen organisat...,Een gids voor het bewerkstelligen van duurzaam...,NF,V,"Green hearts, great business",de magische formule om teams in beweging te kr...,2025,...,NaN,NaN,NaN,Paperback,NaN,NaN,NaN,NaN,auteur,- Elk bedrijf heeft duurzame ambities\t\n- Daa...
1,9789043930413,mas_baf_processed_17284,Een verhelderend boek over borduren. Het boek ...,Een praktische gids over borduren. Het boek bi...,Een praktische gids over borduren. Het boek bi...,NF,V,1000 figuren 365 dagen borduren,een journal met naald en draad,2023,...,NaN,NaN,NaN,Paperback,NaN,NaN,NaN,NaN,auteur,‘1000 figuren 365 dagen borduren’ biedt eindel...
2,9789089657046,mas_baf_processed_15593,"Een boek over persoonlijke ontwikkeling, solli...",NaN,Een boek over ondernemerschap en de balans tus...,NF,V,Relaxed vlammen,"meer misbaar in je bedrijf, minder missen van ...",2023,...,NaN,NaN,NaN,Paperback,NaN,NaN,NaN,NaN,auteur,‘Al je zakelijke doelen behalen met een doorlo...
3,9789057125638,mas_baf_processed_3178,"Een verhelderend boek over gezondheid, vegetar...",Een verhelderend boek over hoe microben in het...,Een verhelderend boek over hoe microben in het...,NF,V,De dieetmythe,de echte wetenschap achter wat we eten,2022,...,NaN,Zweden|Verenigd Koninkrijk|Verenigde Staten|Po...,NaN,Paperback,NaN,NaN,NaN,NaN,geneticus|epidemioloog|non-fictieschrijver,"‘De dieetmythe is fascinerend, en ik ben nu ge..."
4,9789044856323,mas_baf_processed_26167,Een vermakelijk kinderboek. Wanneer de koning ...,Een vermakelijk kinderboek over de eigenwijze ...,Een vermakelijk kinderboek over de eigenwijze ...,F,J,De koning van Babaroetsjka,NaN,2025,...,NaN,Nederland,NaN,Paperback,Avonturen van Glas,0,NaN,NaN,auteur,"‘De koning komt,’ zei de moeder van Julie.\n\t..."


## **Count Placeholders in Texts**

In [3]:
import re

placeholder_re = re.compile(r"\[[^\]]+\]")   # anything in [...]

def count_placeholders(text: str) -> int:
    if pd.isna(text):
        return 0
    return len(placeholder_re.findall(text))

# count per version and a total per row
for col in ["first_version", "second_version", "final_version"]:
    df[f"{col}_ph"] = df[col].map(count_placeholders)

df["total_ph"] = df[[f"{c}_ph" for c in ["first_version","second_version","final_version"]]].sum(axis=1)

print(df["total_ph"].describe())  
print(df["total_ph"].value_counts().sort_index())

# print value counts + percentage 
print(df["total_ph"].value_counts(normalize=True).sort_index() * 100)

count    26210.000000
mean         0.291034
std          1.135419
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          8.000000
Name: total_ph, dtype: float64
total_ph
0    22598
1     2730
2      304
3        4
7      314
8      260
Name: count, dtype: int64
total_ph
0    86.219000
1    10.415872
2     1.159863
3     0.015261
7     1.198016
8     0.991988
Name: proportion, dtype: float64


## **Clean Data**
- 1. Rename columns
- 2. Keep rows with first and final versions present
- 3. Drop rows with too many (> 2) placeholders
- 4. Keep only rows where three versions are distinct

In [5]:
import html

def clean_raw_data(input_path, output_path, verbose=True):
    # Helper
    def log(stage: str, before: int, after: int):
        if verbose:
            removed = before - after
            print(f"{stage:<40} | removed {removed:>6,} | remaining {after:>7,}")

    # Load
    df = pd.read_csv(input_path, sep="\t")
    print(f"Loaded {len(df):,} raw rows")

    # Rename columns
    df = df.rename(
        columns={
            "first_version":  "version1",
            "second_version": "version2",
            "final_version":  "version3",
        }
    )


    # Keep rows with first and final versions present
    before = len(df)
    df = df.dropna(subset=["version1", "version3"])
    log("after dropna (first and final versions present)", before, len(df))

    # Helpers
    def normalise_quotes(text: str) -> str:
        return (
            text.replace("‘", "'").replace("’", "'").replace("‚", "'")
                .replace("“", '"').replace("”", '"').replace("„", '"')
        )

    def clean_text(text) -> str:
        if pd.isna(text):
            return ""
        text = html.unescape(str(text))
        text = normalise_quotes(text)
        text = re.sub(r"\\n|\\t|\n|\t", " ", text)
        text = re.sub(r"\s+", " ", text)
        return text.strip()

    # Apply cleaning
    for col in ["version1", "version2", "version3"]:
        df[col] = df[col].apply(clean_text)

    # Drop rows with *too many* placeholders 
    placeholder = re.compile(r"\[[^\]]+\]")

    def count_ph(text: str) -> int:
        return 0 if pd.isna(text) else len(placeholder.findall(text))

    for col in ["version1", "version2", "version3"]:
        df[f"{col}_ph"] = df[col].map(count_ph)

    df["total_ph"] = df[[f"{c}_ph" for c in
                         ("version1", "version2", "version3")]].sum(axis=1)

    before = len(df)
    MAX_PH = 2
    df = df[df["total_ph"] <= MAX_PH].copy()
    log("after placeholder filter (≤ 2)", before, len(df))

    # clean-up helper cols
    df.drop(columns=[c for c in df.columns
                     if c.endswith("_ph") or c == "total_ph"],
            inplace=True)

    # Keep only rows where the three versions are distinct
    before = len(df)
    df = df[df.apply(
        lambda r: len({r["version1"], r["version2"], r["version3"]}) == 3,
        axis=1)]
    log("after distinct-versions filter", before, len(df))

    # Save
    df[["isbn", "title", "version1", "version2", "version3"]].to_parquet(
        output_path, index=False
    )
    print(f"Saved {len(df):,} clean rows → {output_path}")

    return output_path


# run it
clean_raw_data(config["input_file"], config["cleaned_file"])


C:\Users\kevin\AppData\Local\Temp\ipykernel_1156\3560555693.py:11: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, sep="\t")


Loaded 26,210 raw rows
after dropna (first and final versions present) | removed    261 | remaining  25,949
after placeholder filter (≤ 2)           | removed    578 | remaining  25,371
after distinct-versions filter           | removed    312 | remaining  25,059
Saved 25,059 clean rows → data/clean_books.parquet


'data/clean_books.parquet'